## https://dl.dropboxusercontent.com/u/75194/Databases3.ipynb

## Your Project post milestone 2

- Milestone 2 is due at 11.59PM two mondays from today. Try and finish it earlier though, as you will be hurting your post-milestone project if you dont.
- additional things in milestone 2 will be to implement stored procedures (released wednesday) and predicate selects with orderable indices(released friday) 
- also make sure that writes to our database are atomic

For you project after milestone 2, you must:
- you must make this database persistent. You can choose a persistence of your choice, but one idea is a primary-key clustered index for the primary-key and rows, and bitmask indexes for low-cardinality "factor" or "enum" types which only take certain values and/or binary tree for the numeric ones (you will need to implement low-hi iteration) and/or repeated values for a key . Make sure changes remain atomic, and that they are durable.
- represent vantage points in our persistent database. (this requires writing stored procedures for vantage point calculation and storing distance metadata, and then using the predicate selects. The algorithm is the one Pavlos described in class: choose vantage points proportional to a SAX word based representation. We will talk about this Friday, and do some part of an in-memory implementation).
- there should be a http based REST api to the database. The API design is your choice, and depending on if you decided to write a synchronous client or an asynchronous client, you might want to choose flask or tornado/aiohttp.

This much is compulsory. Additionally, each group can choose to do **one additional feature** of their choice, but let us know what this is by next monday and we can discuss it next monday (no lecture, just a lab, atleast one person from your group should come, but not all need to)

Possible features are (your choice needs to be atleast as hard as this):

1. a really fast FFT using cython on fftw integrated in
2. vantage point storage using a vptree integrated in
3. A multiprocessing based server where multiple processes write to the database. Implement atleast read-committed isolation.
4. implement a proper language with repl as a client for the database
5. your choice (we'll ask the TFs and Pavlos for more ideas if you like)

## Transactions

(most diagrams and all quotes are from Oreilly's Designing Data Intensive Applications: a book you should read if you, like me, enjoynthis stuff).

 •	The batch of operations is viewed as a single atomic operation, so all of the operations either succeed together or fail together.
 •	The database is in a valid state before and after the transaction.
 •	The batch update appears to be isolated; other queries should never see a database state in which only some of the operations have been applied.
 
Databases have a mechanism for wrapping a single or multiple processes into a **Transaction**. This means that the batch of operations either all happen (**commit**) or not happen at all (**abort**, **rollback**). This is called **atomicity**.

The NOSQL movement made transactions into a casualty since they were distributed and worried about replication and partitioning. The notion emerged that distributed-transactions could not be performant. But indeed this notion has been there from even earlier, as we shall see: different guarantee levels in transactions have different performance, even on a single machine, which is the case we focus on in this class.

The transactional guarantees are represented by the acronym `ACID`. The A is for atomicity, which we just described.

* C is for **Consistency**: data invariants must be true. This is really a property of the application: eg accounting tables must be balanced. Databases can help with foreign keys, but this is a property of the app. We wont discuss this one further,
* D is for **Durability**: once a transaction has comitted successfully, data comitted wont be forgotten. This requires persistent storage, or replication, or both
* I is for **Isolation**. This is the most interesting of the lot, and critical to the sensible running of a databse. The idea is that transactions should not step on each other. Each transaction should pretends that its the only one running on the databse: in other words, as if the transactions wer completely serialized. In practice this would make things very slow, so we try different transactional guarantees that fall short of explicit serialization except in the situations that really need serialization.

Atomicity and Isolation are usually provided by most databases for single object writes, like a single set (or for that matter a single get) on one machine. As we shall see later, these are critical to prevent "lost updates" and are a big part of the transaction story, one that is implemented even in the new-fangled databases. But they are not the full story. We need multi-object transactionsto

- deal with foreign keys
- deal with the denormalization seen in nosql's like mongo
- deal with secondary indexes.

Lets look at all of these scenarios using two examples: a relational setup, and our append based binary-tree key-value database or index.

In [1]:
class ValueRef(object):
    " a reference to a string value on disk"
    def __init__(self, referent=None, address=0):
        self._referent = referent #value to store
        self._address = address #address to store at
        
    @property
    def address(self):
        return self._address
    
    def prepare_to_store(self, storage):
        pass

    @staticmethod
    def referent_to_bytes(referent):
        return referent.encode('utf-8')

    @staticmethod
    def bytes_to_referent(bytes):
        return bytes.decode('utf-8')

    
    def get(self, storage):
        "read bytes for value from disk"
        if self._referent is None and self._address:
            self._referent = self.bytes_to_referent(storage.read(self._address))
        return self._referent

    def store(self, storage):
        "store bytes for value to disk"
        #called by BinaryNode.store_refs
        if self._referent is not None and not self._address:
            self.prepare_to_store(storage)
            self._address = storage.write(self.referent_to_bytes(self._referent))




In [2]:
import pickle
class BinaryNodeRef(ValueRef):
    "reference to a btree node on disk"
    
    #calls the BinaryNode's store_refs
    def prepare_to_store(self, storage):
        "have a node store its refs"
        if self._referent:
            self._referent.store_refs(storage)

    @staticmethod
    def referent_to_bytes(referent):
        "use pickle to convert node to bytes"
        return pickle.dumps({
            'left': referent.left_ref.address,
            'key': referent.key,
            'value': referent.value_ref.address,
            'right': referent.right_ref.address,
        })

    @staticmethod
    def bytes_to_referent(string):
        "unpickle bytes to get a node object"
        d = pickle.loads(string)
        return BinaryNode(
            BinaryNodeRef(address=d['left']),
            d['key'],
            ValueRef(address=d['value']),
            BinaryNodeRef(address=d['right']),
        )
    


In [3]:
class BinaryNode(object):
    @classmethod
    def from_node(cls, node, **kwargs):
        "clone a node with some changes from another one"
        return cls(
            left_ref=kwargs.get('left_ref', node.left_ref),
            key=kwargs.get('key', node.key),
            value_ref=kwargs.get('value_ref', node.value_ref),
            right_ref=kwargs.get('right_ref', node.right_ref),
        )

    def __init__(self, left_ref, key, value_ref, right_ref):
        self.left_ref = left_ref
        self.key = key
        self.value_ref = value_ref
        self.right_ref = right_ref

    def store_refs(self, storage):
        "method for a node to store all of its stuff"
        self.value_ref.store(storage)
        #calls BinaryNodeRef.store. which calls
        #BinaryNodeRef.prepate_to_store
        #which calls this again and recursively stores
        #the whole tree
        self.left_ref.store(storage)
        self.right_ref.store(storage)

In [19]:
class BinaryTree(object):
    "Immutable Binary Tree class. Constructs new tree on changes"
    def __init__(self, storage):
        self._storage = storage
        self._refresh_tree_ref()

    def commit(self):
        "changes are final only when committed"
        #triggers BinaryNodeRef.store
        self._tree_ref.store(self._storage)
        #make sure address of new tree is stored
        self._storage.commit_root_address(self._tree_ref.address)

    def _refresh_tree_ref(self):
        "get reference to new tree if it has changed"
        self._tree_ref = BinaryNodeRef(
            address=self._storage.get_root_address())

    def get(self, key):
        "get value for a key"
        #your code here
        #if tree is not locked by another writer
        #refresh the references and get new tree if needed
        if not self._storage.locked:
            self._refresh_tree_ref()
        #get the top level node
        node = self._follow(self._tree_ref)
        #traverse until you find appropriate node
        while node is not None:
            if key < node.key:
                node = self._follow(node.left_ref)
            elif key < node.key:
                node = self._follow(node.right_ref)
            else:
                return self._follow(node.value_ref)
        raise KeyError

    def set(self, key, value):
        "set a new value in the tree. will cause a new tree"
        #try to lock the tree. If we succeed make sure
        #we dont lose updates from any other process
        if self._storage.lock():
            self._refresh_tree_ref()
        #get current top-level node and make a value-ref
        node = self._follow(self._tree_ref)
        value_ref = ValueRef(value)
        #insert and get new tree ref
        self._tree_ref = self._insert(node, key, value_ref)
        
    
    def _insert(self, node, key, value_ref):
        "insert a new node creating a new path from root"
        #create a tree ifnthere was none so far
        if node is None:
            new_node = BinaryNode(
                BinaryNodeRef(), key, value_ref, BinaryNodeRef())
        elif key < node.key:
            new_node = BinaryNode.from_node(
                node,
                left_ref=self._insert(
                    self._follow(node.left_ref), key, value_ref))
        elif key > node.key:
            new_node = BinaryNode.from_node(
                node,
                right_ref=self._insert(
                    self._follow(node.right_ref), key, value_ref))
        else: #create a new node to represent this data
            new_node = BinaryNode.from_node(node, value_ref=value_ref)
        return BinaryNodeRef(referent=new_node)

    def delete(self, key):
        "delete node with key, creating new tree and path"
        if self._storage.lock():
            self._refresh_tree_ref()
        node = self._follow(self._tree_ref)
        self._tree_ref = self._delete(node, key)
        
    def _delete(self, node, key):
        "underlying delete implementation"
        if node is None:
            raise KeyError
        elif key < node.key:
            new_node = BinaryNode.from_node(
                node,
                left_ref=self._delete(
                    self._follow(node.left_ref), key))
        elif key > node.key:
            new_node = BinaryNode.from_node(
                node,
                right_ref=self._delete(
                    self._follow(node.right_ref), key))
        else:
            left = self._follow(node.left_ref)
            right = self._follow(node.right_ref)
            if left and right:
                replacement = self._find_max(left)
                left_ref = self._delete(
                    self._follow(node.left_ref), replacement.key)
                new_node = BinaryNode(
                    left_ref,
                    replacement.key,
                    replacement.value_ref,
                    node.right_ref,
                )
            elif left:
                return node.left_ref
            else:
                return node.right_ref
        return BinaryNodeRef(referent=new_node)

    def _follow(self, ref):
        "get a node from a reference"
        #calls BinaryNodeRef.get
        return ref.get(self._storage)
    
    def _find_max(self, node):
        while True:
            next_node = self._follow(node.right_ref)
            if next_node is None:
                return node
            node = next_node

In [34]:
import os
import struct

import portalocker


class Storage(object):
    SUPERBLOCK_SIZE = 4096
    INTEGER_FORMAT = "!Q"
    INTEGER_LENGTH = 8

    def __init__(self, f):
        self._f = f
        self.locked = False
        #we ensure that we start in a sector boundary
        self._ensure_superblock()

    def _ensure_superblock(self):
        "guarantee that the next write will start on a sector boundary"
        self.lock()
        self._seek_end()
        end_address = self._f.tell()
        if end_address < self.SUPERBLOCK_SIZE:
            self._f.write(b'\x00' * (self.SUPERBLOCK_SIZE - end_address))
        self.unlock()

    def lock(self):
        "if not locked, lock the file for writing"
        if not self.locked:
            portalocker.lock(self._f, portalocker.LOCK_EX)
            self.locked = True
            return True
        else:
            return False

    def unlock(self):
        if self.locked:
            self._f.flush()
            portalocker.unlock(self._f)
            self.locked = False

    def _seek_end(self):
        self._f.seek(0, os.SEEK_END)

    def _seek_superblock(self):
        "go to beginning of file which is on sec boundary"
        self._f.seek(0)

    def _bytes_to_integer(self, integer_bytes):
        return struct.unpack(self.INTEGER_FORMAT, integer_bytes)[0]

    def _integer_to_bytes(self, integer):
        return struct.pack(self.INTEGER_FORMAT, integer)

    def _read_integer(self):
        return self._bytes_to_integer(self._f.read(self.INTEGER_LENGTH))

    def _write_integer(self, integer):
        self.lock()
        self._f.write(self._integer_to_bytes(integer))

    def write(self, data):
        "write data to disk, returning the adress at which you wrote it"
        #first lock, get to end, get address to return, write size
        #write data, unlock <==WRONG, dont want to unlock here
        #your code here
        self.lock()
        self._seek_end()
        object_address = self._f.tell()
        self._write_integer(len(data))
        self._f.write(data)
        return object_address

    def read(self, address):
        self._f.seek(address)
        length = self._read_integer()
        data = self._f.read(length)
        return data

    def commit_root_address(self, root_address):
        self.lock()
        self._f.flush()
        #make sure you write root address at position 0
        self._seek_superblock()
        #write is atomic because we store the address on a sector boundary.
        self._write_integer(root_address)
        self._f.flush()
        self.unlock()

    def get_root_address(self):
        #read the first integer in the file
        #your code here
        self._seek_superblock()
        root_address = self._read_integer()
        return root_address

    def close(self):
        self.unlock()
        self._f.close()

    @property
    def closed(self):
        return self._f.closed

In [21]:
class DBDB(object):

    def __init__(self, f):
        self._storage = Storage(f)
        self._tree = BinaryTree(self._storage)

    def _assert_not_closed(self):
        if self._storage.closed:
            raise ValueError('Database closed.')

    def close(self):
        self._storage.close()

    def commit(self):
        self._assert_not_closed()
        self._tree.commit()

    def get(self, key):
        self._assert_not_closed()
        return self._tree.get(key)

    def set(self, key, value):
        self._assert_not_closed()
        return self._tree.set(key, value)

    def delete(self, key):
        self._assert_not_closed()
        return self._tree.delete(key)

In [22]:
def connect(dbname):
    try:
        f = open(dbname, 'r+b')
    except IOError:
        fd = os.open(dbname, os.O_RDWR | os.O_CREAT)
        f = os.fdopen(fd, 'r+b')
    return DBDB(f)

In [23]:
!rm /tmp/test2.dbdb

In [24]:

db = connect("/tmp/test2.dbdb")

In [25]:
db.set("rahul", "aged")
db.set("pavlos", "aged")
db.set("kobe", "stillyoung")

In [26]:
db.close()

In [27]:
db = connect("/tmp/test2.dbdb")
db.get("rahul")

KeyError: 

In [28]:
db.set("rahul", "aged")
db.set("pavlos", "aged")
db.set("kobe", "stillyoung")
db.commit()

In [29]:
db.close()
db = connect("/tmp/test2.dbdb")
db.get("rahul")

'aged'

In [30]:
db.set("rahul", "young")
db.get("rahul")

'young'

In [31]:
db.close()
db = connect("/tmp/test2.dbdb")
db.get("rahul")

'aged'

In [32]:
db.set("rahul", "young")
db.commit()
db.close()
db = connect("/tmp/test2.dbdb")
db.get("rahul")

'young'

In [33]:
db.delete("pavlos")
db.commit()

The critical parts of the code from above are these:

from Storage:

```python
    def commit_root_address(self, root_address):
        self.lock()
        self._f.flush()
        #make sure you write root address at position 0
        self._seek_superblock()
        #write is atomic because we store the address on a sector boundary.
        self._write_integer(root_address)
        self._f.flush()
        self.unlock()
        
    def _ensure_superblock(self):
        "guarantee that the next write will start on a sector boundary"
        self.lock()
        self._seek_end()
        end_address = self._f.tell()
        if end_address < self.SUPERBLOCK_SIZE:
            self._f.write(b'\x00' * (self.SUPERBLOCK_SIZE - end_address))
        self.unlock()
```

get and set on the BinaryTree:

```python
    def get(self, key):
        "get value for a key"
        #your code here
        #if tree is not locked by another writer
        #refresh the references and get new tree if needed
        if not self._storage.locked:
            self._refresh_tree_ref()
        #get the top level node
        node = self._follow(self._tree_ref)
        #traverse until you find appropriate node
        while node is not None:
            if key < node.key:
                node = self._follow(node.left_ref)
            elif key < node.key:
                node = self._follow(node.right_ref)
            else:
                return self._follow(node.value_ref)
        raise KeyError

    def set(self, key, value):
        "set a new value in the tree. will cause a new tree"
        #try to lock the tree. If we succeed make sure
        #we dont lose updates from any other process
        if self._storage.lock():
            self._refresh_tree_ref()
        #get current top-level node and make a value-ref
        node = self._follow(self._tree_ref)
        value_ref = ValueRef(value)
        #insert and get new tree ref
        self._tree_ref = self._insert(node, key, value_ref)
        
    def _write_integer(self, integer):
        self.lock()
        self._f.write(self._integer_to_bytes(integer))

    def write(self, data):
        "write data to disk, returning the adress at which you wrote it"
        #first lock, get to end, get address to return, write size
        #write data, unlock
        #your code here
        self.lock()
        self._seek_end()
        object_address = self._f.tell()
        self._write_integer(len(data))
        self._f.write(data)
        return object_address
```


Writing out the address of the root node ia atomic since we are on a disk sector boundary, and this is the very first thing in the file. The disk-hardware guarantees that single-sector writes are atomic. 

The above case is atomic in the single-object sense because the changes are made and comitted, creating a new version of the tree. If things fail, the tree wont be created and you are back to the previous version. We havent explicitly built in any abort-handling though, replying on the python exceptions to do this for us. Another process will NEVER see a mix of writes: the root address will either be the old value or the new one. 

Our very own in memory process, will, however, reflect the partial changes we made, so a proper transaction manager should kill the "thread" of execution. But we havent talked about a process model yet, and will come to this the next time. Today we'll assume we have different processes (so that we dont have to worry about the GIL in thinking about any of this).

We also write the new data (the new tree) to disk (and flush) before we update the root address, there is no way to get at them as yet. When the root address is updated, we are guaranteed its data is now on disk. This ensures durability: once the transaction is comitted, its dats are on disk, and until it is comitted, this data is not reachable, indeed only the old data is (even if the new data is eating up pprecious bytes).

But what about isolation?

## Why is isolation important?

It is hard to program without isolation. isolation is what guarantees stability. It is what makes sure that there are no dirty reads and dirty writes.

From designing data intensive applications:

Dirty reads

>One client reads another client’s writes before they have been committed. The read committed isolation level and stronger levels prevent dirty reads.


Dirty writes

>One client overwrites data that another client has written, but not yet committed. Almost all transaction implementations prevent dirty writes.

Clearly, the notions of isolation are really the notions of concurrenvy: these issues will also occut when2 programs access any data, in mempry or in a database. In both cases locks and other ideas must be used to make sure that there is only one mutator at a time, and that an object is not exposed in an inconsistent state.

### Read Committed

No dirty reads and no dirty writes.

Dirty reads means that you can peek into the goings on inside a transaction and get at itermediate values. This could mean you see a value that would be later rolled back.

with dirty writes you could be overwriting an uncomitted value

![](https://dl.dropboxusercontent.com/u/75194/dirtywrite.png)

In Read Comitted mode, in essence we need to remember the pre-transaction value, ie the old comitted value and the also the new value being set by the transaction which holds the current write lock. While this transaction runs, any other thread/transaction sees the old value. Once this one commits, then the values read by the other transaction are switched. And once the lock is held, no-one else can write.

You can see this in the `get` above where we check whether any other transaction is running and if not get the latest tree. In a regular database we similarly use row-level locks.Another writer must wait until the lock is given up, and the reader will read the value before the lock was set as long as the write transaction is in progress.

So if we implemented a transaction system for our database, the current code would put it in Read Committed mode.

### The Read Skew Problem

>Non-repeatable reads:
>A client sees different parts of the database at different points in time. This is most commonly prevented with snapshot isolation, which allows a transaction to read from a consistent snapshot at one point in time. It is usually implemented with multi-version concurrency control (MVCC).

![](https://dl.dropboxusercontent.com/u/75194/readskew.png)

If you look at the diagram above, it makes perfect sence if Alice's two reads are separate read transactions. But what if they are one transaction and we are in read-comitted mode. Since the reads took a very long time and were on both sides of a kosher transaction, they might give Alice a heart attack. This is a non-repeatable read as if Alice were to re-issue the reas transaction, acct 1 would now show 600. This problem is perfectly consistent with operations in read-comitted mode: the second read gets the switched value in analog to our get.

The fix for this would seem to be clear: if both the reads happen in one transaction, one must make sure that because this transaction happened before the write, it sees the older version of the accounts (500, 500). Cast into the language of our database, the check we do for a get ought to be suspended if the get is inside an already started transaction. In general, the only place a tree-refresh ought to be allowed is at the beginning of a read-or write transaction.

These thoughts form the basis of Multi-Version concurrency control, or MVCC, also known as snapshot isolation.

The key principle is (from DDIA):
> readers never block writers, and writers never block readers

Here then are the rules:

>1. At the start of each transaction, the database makes a list of all the other transac‐ tions which are in progress (not yet committed or aborted) at that time. Any writes made by one of those transactions are ignored, even if the transaction sub‐ sequently commits.
2. Any writes made by aborted transactions are ignored.
3. Any writes made by transactions with a later transaction ID (i.e. which started after the current transaction started) are ignored, regardless of whether that transaction has committed.
4. All other writes are visible to the application’s queries.

Here's what snapshot isolation using MVCC looks like:

![](https://dl.dropboxusercontent.com/u/75194/mvcc.png)

What about writes?

There is another problem we encounter here: 

### the lost update problem.

Consider an upsert, where we read a value, do something with it, like the balance change above, and then write it. Because we only lock writes, a transaction T1 might read a value V0, and then transaction T2 reads value V0. Since T2 does not have lock, the code in `get` wont get a new tree, and uses the old value. This is good as the transaction T1 is still running.

But here is the problem: T1 will create a new tree, say with V0+1. Now T2 comes along, operates on V0, and makes it V0+2. say V0 was 5 to start with. The notion of the process might have been 5->6->8. But we wont do that. Our latest value will be 7.

![](https://dl.dropboxusercontent.com/u/75194/lostupdate.png)

The fix to this is atomic updates/upserts. The `UPDATE` command in sql does just that, and if we build transactional facility into our database to do multiple gets, there is no reason not to extend it to an upsert. The entire upsert would grab the lock, and once that happens this would effectively serialize T1 and T2. Atomic updates are usually thus built by taking the lock when the object is read, or by forcing all atomic operations to be on one given thread.

So thus it seems we can add a transaction manager, have an explicit notion of read, write, and upsert transactions, and have a reasonable snapshot isolated database.

But even this does not solve some problems

### Write skew

![](https://dl.dropboxusercontent.com/u/75194/writeskew.png)

Imagine the following situation: you and i both concurrently try and book the same time slot. you go first and book it. I start before you. and you start next. We both check to see if there is a booking at this time, and there is not. If each booking is a different object, then we both can make the booking: this correspondsto set on two separate keys which have an application level commonality in meaning but not necessarily at the database level. Snapshot isolation will happily allow this.

Notice this is a multi object transaction with a select based on some predicate and stuff changes under the second select.

This would only work if the transactions were fully serialized, as then the second bookers check would wait on the first bookers transaction being completed and thus raise the conflict. 

> This effect, where a write in one transaction changes the result of a search query in another transaction, is called a phantom. Snapshot isolation avoids phantoms in read-only queries, but in read-write transactions like the examples we discussed, phantoms can lead to particularly tricky cases of write skew.

One could solve this by having a timeslot-room key but now you are letting concurrency dictate your application model. The most general solution is to use serialized isolation.

### Serialized Isolation

- one could actually run things in serial on one thread. With in memory databases this has become faster. Voltdb/h-store does this with replication, and stored procedures (in java and groovy, a JVM language).

#### stored procedures

Stored procedures run inside the database address space, the idea being to eliminate the overhead of serialization and de-serialization, network transfer, etc (but a badly written stored procedure could be worse)

#### multithreaded

but if you want multithreaded, the old, but not gold since its slowness makes people not like it is 2-phase locking(2PL). 

- the ingredients are shared read locks and exclusive write locks.

The process then is:

>- if a transaction wants to read an object, it must first acquire the lock in shared mode. Several transactions are allowed to hold the lock in shared mode simultaneously, but if another transaction already has an exclusive lock on the object, the transaction must wait.
- If a transaction wants to write to an object, it must first acquire the lock in exclu‐ sive mode. No other transaction may hold the lock at the same time (neither in shared nor in exclusive mode), so if there is any existing lock on the object, the transaction must wait.
- If a transaction first reads and then writes an object, it may upgrade its shared lock to an exclusive lock. The upgrade works the same as getting an exclusive lock directly.
- After a transaction has acquired the lock, it must continue to hold the lock until the end of the transaction (commit or abort). This is where the name “two- phase” comes from: the first phase (while the transaction is executing) is when the locks are acquired, and the second phase (at the end of the transaction) is when all the locks are released.

The solution to this in our database has the flavor of serialized upserts, except that we now even want to serialize the 2 separate sets. So then, reading the bookings, the get, would have a shared lock. If T1 has an exclusive lock to write in there, everyone else must wait on it to complete. At this point T2 cant even read until T1 is complete, so when T2 runs, it gets the latest info. Now T2 can write.

#### Predicate and Range Locks

Now to fix the room booking problem, we want to do  a predicate lock on the "key", here a set of time bins that we wish to occupy. You can imagine the conditional checking after pulling from the database is an expensive operation. Thus we can lock a range of keys instead.